# Interest-rate models #
## 14. Short Interest Rate Model Calibration ##

In [1]:
import QuantLib as ql
from collections import namedtuple
import math
import pandas as pd

### Hull-White 1 Factor Model ###

In [2]:
today = ql.Date(15, ql.February, 2002);
settlement= ql.Date(19, ql.February, 2002);
ql.Settings.instance().evaluationDate = today;
term_structure = ql.YieldTermStructureHandle(ql.FlatForward(settlement, 0.04875825, ql.Actual365Fixed()))
index = ql.Euribor1Y(term_structure)

In [3]:
CalibrationData = namedtuple("CalibrationData", "start, length, volatility")
data = [CalibrationData(1, 5, 0.1148),
        CalibrationData(2, 4, 0.1108),
        CalibrationData(3, 3, 0.1070),
        CalibrationData(4, 2, 0.1021),
        CalibrationData(5, 1, 0.1000 )]

In [4]:
def create_swaption_helpers(data, index, term_structure, engine):
    swaptions = []
    fixed_leg_tenor = ql.Period(1, ql.Years)
    fixed_leg_daycounter = ql.Actual360()
    floating_leg_daycounter = ql.Actual360()
    for d in data:
        vol_handle = ql.QuoteHandle(ql.SimpleQuote(d.volatility))
        helper = ql.SwaptionHelper(ql.Period(d.start, ql.Years), 
                                   ql.Period(d.length, ql.Years),
                                   vol_handle, index, fixed_leg_tenor,
                                   fixed_leg_daycounter, floating_leg_daycounter,
                                   term_structure)
        helper.setPricingEngine(engine)
        swaptions.append(helper)
    return swaptions

In [5]:
def calibration_report(swaptions, data):
    columns = ["Model Price", "Market Price", "Implied Vol", "Market Vol", "Rel Error Price", "Rel Error Vols"]
    report_data = []
    cum_err = 0.0
    cum_err2 = 0.0
    for i, s in enumerate(swaptions):
        model_price = s.modelValue()
        market_vol = data[i].volatility
        black_price = s.blackPrice(market_vol)
        rel_error = model_price / black_price - 1.0
        implied_vol = s.impliedVolatility(model_price, 1e-5, 50, 0.0, 0.50)
        rel_error2 = implied_vol / market_vol - 1.0
        cum_err += rel_error * rel_error
        cum_err2 += rel_error2 * rel_error2
        report_data.append((model_price, black_price, implied_vol, market_vol, rel_error, rel_error2))
    print("Cumulative Error Price: %7.5f" % math.sqrt(cum_err))
    print("Cumulative Error Vols : %7.5f" % math.sqrt(cum_err2))
    return pd.DataFrame(report_data, columns=columns, index=[''] * len(report_data))

### Calibration Reversion and Volatility ###

In [6]:
model = ql.HullWhite(term_structure);
engine = ql.JamshidianSwaptionEngine(model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = ql.LevenbergMarquardt(1.0e-8, 1.0e-8, 1.0e-8)
end_criteria = ql.EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma = model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.04642, sigma = 0.00580


In [7]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.11583
Cumulative Error Vols : 0.11614


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008775,0.009485,0.106198,0.1148,-0.074854,-0.074928
,0.009669,0.010078,0.106292,0.1108,-0.040610,-0.040688
,0.008663,0.008716,0.106343,0.1070,-0.006138,-0.006138
,0.006490,0.006226,0.106442,0.1021,0.042367,0.042525
,0.003542,0.003323,0.106612,0.1000,0.065817,0.066122


### Calibrating Volatility With Fixed Reversion ###

In [8]:
constrained_model = ql.HullWhite(term_structure, 0.05, 0.001);
engine = ql.JamshidianSwaptionEngine(constrained_model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = ql.LevenbergMarquardt(1.0e-8, 1.0e-8, 1.0e-8)
end_criteria = ql.EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
constrained_model.calibrate(swaptions, optimization_method, end_criteria, ql.NoConstraint(), [], [True, False])
a, sigma = constrained_model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.05000, sigma = 0.00586


In [9]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.11584
Cumulative Error Vols : 0.11615


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008776,0.009485,0.106212,0.1148,-0.074738,-0.074812
,0.009668,0.010078,0.106284,0.1108,-0.040682,-0.040761
,0.008662,0.008716,0.106330,0.1070,-0.006261,-0.006261
,0.006490,0.006226,0.106436,0.1021,0.042311,0.042469
,0.003542,0.003323,0.106625,0.1000,0.065946,0.066252


### Black Karasinski Model ###

In [10]:
model = ql.BlackKarasinski(term_structure);
engine = ql.TreeSwaptionEngine(model, 100)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = ql.LevenbergMarquardt(1.0e-8, 1.0e-8, 1.0e-8)
end_criteria = ql.EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma = model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.03949, sigma = 0.11678


In [11]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.12382
Cumulative Error Vols : 0.12415


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008702,0.009485,0.105306,0.1148,-0.082623,-0.082704
,0.009669,0.010078,0.106291,0.1108,-0.040617,-0.040695
,0.008670,0.008716,0.106429,0.1070,-0.005334,-0.005334
,0.006502,0.006226,0.106636,0.1021,0.044257,0.044423
,0.003555,0.003323,0.107010,0.1000,0.069778,0.070104


### G2++ Model ###

In [12]:
model = ql.G2(term_structure);
engine = ql.TreeSwaptionEngine(model, 25)
# engine = ql.G2SwaptionEngine(model, 10, 400)
# engine = ql.FdG2SwaptionEngine(model)
swaptions = create_swaption_helpers(data, index, term_structure, engine)
optimization_method = ql.LevenbergMarquardt(1.0e-8, 1.0e-8, 1.0e-8)
end_criteria = ql.EndCriteria(1000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma, b, eta, rho = model.params()
print("a = %6.5f, sigma = %6.5f, b = %6.5f, eta = %6.5f, rho = %6.5f " % (a, sigma, b, eta, rho))

a = 0.04758, sigma = 0.00305, b = 0.03850, eta = 0.00478, rho = -0.00000 


In [13]:
calibration_report(swaptions, data)

Cumulative Error Price: 0.12410
Cumulative Error Vols : 0.12442


,Model Price,Market Price,Implied Vol,Market Vol,Rel Error Price,Rel Error Vols
,0.008696,0.009485,0.105235,0.1148,-0.083235,-0.083316
,0.009671,0.010078,0.106320,0.1108,-0.040359,-0.040437
,0.008666,0.008716,0.106389,0.1070,-0.005712,-0.005712
,0.006500,0.006226,0.106604,0.1021,0.043950,0.044115
,0.003555,0.003323,0.107018,0.1000,0.069858,0.070184


### Calibrating to Normal Volatilities ###

In [16]:
def create_swaption_helpers_normal(data, index, term_structure, engine):
    swaptions = []
    fixed_leg_tenor = ql.Period(1, ql.Years)
    fixed_leg_daycounter = ql.Actual360()
    floating_leg_daycounter = ql.Actual360()
    for d in data:
        vol_handle = ql.QuoteHandle(ql.SimpleQuote(d.volatility))
        helper= ql.SwaptionHelper(ql.Period(d.start, ql.Years),
                                  ql.Period(d.length, ql.Years),
                                  vol_handle, index, fixed_leg_tenor,
                                  fixed_leg_daycounter, floating_leg_daycounter,
                                  term_structure, ql.BlackCalibrationHelper.RelativePriceError,
                                  ql.nullDouble(), 1.0, ql.Normal)
        helper.setPricingEngine(engine)
        swaptions.append(helper)
    return swaptions

In [17]:
model = ql.HullWhite(term_structure);
engine = ql.JamshidianSwaptionEngine(model)
swaptions = create_swaption_helpers_normal(data, index, term_structure, engine)
optimization_method = ql.LevenbergMarquardt(1.0e-8, 1.0e-8, 1.0e-8)
end_criteria = ql.EndCriteria(10000, 100, 1e-6, 1e-8, 1e-8)
model.calibrate(swaptions, optimization_method, end_criteria)
a, sigma = model.params()
print("a = %6.5f, sigma = %6.5f" % (a, sigma))

a = 0.00003, sigma = 0.10459
